# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [74]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [75]:
# !cat talks.tsv

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [76]:
talks = pd.read_csv("talks_new.csv")
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Problems and Ideas Relating to Sharing Code in...,Talk,ASB-Student-Seminar,American Society of Biomechanics Student Semin...,2022-12-01,Virtual,https://www.youtube.com/watch?v=D6GV7sVvbsA&t=0s,Oral presentation by Mitchell Tillman
1,sciOS a ”science operating system”: Developing...,Seminar,Dept-Seminar,Department Seminar,2022-02-02,Hoboken NJ USA,NaN,Oral presentation by Mitchell Tillman
2,The Role of Each Leg In Turn Performance,Seminar,Dept-Seminar,Department Seminar,2022-02-01,Hoboken NJ USA,NaN,Oral presentation by Mitchell Tillman
3,The Biomechanics of Frontal Plane Balance,Seminar,Dept-Seminar,Department Seminar,2021-09-01,Hoboken NJ USA,NaN,Oral presentation by Mitchell Tillman
4,Momenta control during each phase of gait reve...,Poster,SFN,Society for Neuroscience,2023-11-01,Washington D.C. USA,NaN,Poster presentation by Mitchell Tillman
5,Frontal Plane Balance Patterns of Older Adults...,Poster,ASB,American Society of Biomechanics,2023-08-01,Knoxville TN USA,NaN,Poster presentation by Zahava Hirsch
6,Gait Phase-Specific Linear and Angular Momentu...,Poster,ISPGR,International Society of Posture and Gait Rese...,2023-07-01,Virtual Brisbane AU,NaN,Poster presentation by Mitchell Tillman
7,Evaluating the Understandability of Real-Time ...,Poster,SB3C,Summer Biomechanics Bioengineering and Biotran...,2023-06-01,Vail CO USA,NaN,Poster presentation by Vibha Iyer
8,Generation of Transverse Plane Linear and Angu...,Podium,ASB-NACOB,American Society of Biomechanics & North Ameri...,2022-08-01,Ottawa CA,NaN,Podium presentation by Mitchell Tillman
9,Lateral Distance as a Measure of Balance Durin...,Podium,SB3C,Summer Biomechanics Bioengineering and Biotran...,2022-06-01,Cambridge MA USA,NaN,Podium presentation by Dr. Antonia Zaferiou. S...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [77]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [81]:
talk_url_str_dict = {}
talk_url_str_dict["Talk"] = "Watch the talk here"
# talk_url_str_dict["Poster"] = "Download the poster here"

loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"    
    
    if len(str(item.talk_url)) > 3: # Has URL
        md += f"\n[{talk_url_str_dict[item.type]}](" + item.talk_url + ")\n"
    elif os.path.exists(os.path.join(os.path.dirname(os.getcwd()),'files',item.type.lower(),md_filename.replace('.md','.pdf'))): # Has file        
        item.talk_url = f"www.mtillman14.github.io/files/{item.type.lower()}/{md_filename.replace('.md','.pdf')}"
        md += f"\n[Download here]({item.talk_url})\n"
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [79]:
!ls ../_talks

2017-02-01-ASB.md                 2021-09-01-Dept-Seminar.md
2017-04-01-Purdue.md              2021-10-01-EMBC.md
2017-05-01-Purdue.md              2022-02-01-Dept-Seminar.md
2017-07-01-PMC.md                 2022-02-02-Dept-Seminar.md
2017-08-01-ASB.md                 2022-04-01-NEBEC.md
2018-04-01-Purdue.md              2022-06-01-SB3C.md
2018-08-01-ASB.md                 2022-08-01-ASB-NACOB.md
2019-02-01-ASB.md                 2022-12-01-ASB-Student-Seminar.md
2019-08-01-ASB.md                 2023-06-01-SB3C.md
2019-08-02-ASB.md                 2023-07-01-ISPGR.md
2019-10-01-SFN.md                 2023-08-01-ASB.md
2020-08-01-ASB.md                 2023-11-01-SFN.md
2021-08-01-ASB.md


In [80]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
